# Iterative Prisoner's Dilemma


## Description

The [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma) (PD) is a classical game analyzed in game theory, which is widely used to (attempt to) model social/economical interaction. It's a "dilemma" as, if exploited to explain the emergence of altruism in human or in general animal society, it fails badly at a first glance.

The classical situation-representation of the PD is that of two prisoners whose conviction depends on their mutual cooperation. It is easier understood though if illustrated in terms of a trade-off game (closed bag exachange):

*Two people meet and exchange closed bags, with the understanding that one of them contains money, and the other contains a purchase. Either player can choose to honor the deal by putting into his or her bag what he or she agreed, or he or she can defect by handing over an empty bag.*

It is obvious that for both players the winning strategy is to NOT cooperate.

Things changes when the interaction between the two individuals is iterated, in that case a more altruist attitude (strategy) is expected to emerge. The goal of this project is to test this hypothesis.

Mathematically the PD can be expressed with very basic linear algebra. The key component is the **Payoff matrix** $M$, which quantify the reward each player gets depending on whether she cooperated or not (defect):

$$
M = 
\begin{pmatrix} 
R & S \\
T & P 
\end{pmatrix}
$$

with $T,R,S,P$ integers that satisfy the following conditions:

$$
T>R>P>S; \quad 2R > T+S
$$

for example $T=3$, $R=2$, $P=1$ and $S=0$, or  $T=5$, $R=3$, $P=2$, $S=0$. Each player choice (move) can be represented by one of the two axis in ${\rm I\!R}^2$, i.e. $u_C=\begin{pmatrix} 1 \\ 0 \end{pmatrix}$ or $u_D=\begin{pmatrix} 0 \\ 1 \end{pmatrix}$, where the first coordinate stands for *Cooperate* and the second for *Defect*. Being $u_1$ and $u_2$ their rewards $r_1$ and $r_2$ can be computed then as:

$$
r_1 = u_1^T M u_2
\quad
\quad
r_2 = u_2^T M u_1
$$

In an Iterative Prisoner's Dilemma (IPD), two players play prisoner's dilemma more than once in succession and they remember previous actions of their opponent and change their strategy accordingly. The winning strategy is the one which yields to a larger reward at the end of the IPD.

The strategy can be represented as a function which outputs either $u_C$ or $u_D$. Such function can depend on the opponent's history of moves, her on history of moves, on the number of moves played till that moment and so on, but it can only be based on a probability density function. Possible strategies are:

* **Nice guy**: always cooperate (the function's output is always $u_D$)
* **Bad guy**: always defect 
* **Mainly nice**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k<50$
* **Mainly bad**: randomly defect $k\%$ of the times and cooperate $100-k\%$, $k>50$
* **tit-for-tat**: start by cooperating, then repeat what the opponent has done in the previous move 

Many more and much more complex strategies can be implemented. The strategy can even change during the IPD.


## Assignments

* Implement a simple IPD between two players implementing two given strategies. Study the evolution along the tournament confronting different strategies; study the overall outcome in the different configurations. 
* Implement a multiple players IPD (MPIPD) where several strategies play against each other in a roud-robin scheme
* Iterate what done in the previous task (repeated MPIPD, rMPIPD)  by increasing the population implementing a given strategy depending on the results that strategy achieved in the previous iteration
* Implement a rMPIPD where strategies are allowed to mutate. The goal is to simulate the effect of genetic mutations and the effect of natura selection. A parameter (gene) should encode the attidue of an individual to cooperate, such gene can mutate randomly and the corresponding phenotype should compete in the MPIPD such that the best-fitted is determined.

In [111]:
import numpy as np
import pandas as pd
import random

## Constants
Let's define the constants needed for the assignement

In [112]:
# Definition of the matrix of the payoff
M = np.array([[2, 0], [3, 1]])

# Definition of the two possible moves
uc = np.array([1, 0])
ud = np.array([0, 1])

print('The matrix of the payoff')
print(M)
# print(np.dot(np.dot(uc.T, M), uc))

# Definition of the probability for defining the two moves
probability_mainly_nice = 0.75
probability_mainly_bad = 0.75

# Definition of the number of stage in the repeated game
n = 1000

The matrix of the payoff
[[2 0]
 [3 1]]


## Definition of the strategies

Disclamer, game theoretic speaking, this is not the formal way to define strategies. In fact in game theory all player decide in advance what move to play in every single circumstance, here we only care about what happenes on the moves' path.

In [113]:
def nice_guy(number_of_stage):
	strat = uc.copy()
	return np.broadcast_to(strat, (number_of_stage, 2))

def bad_guy(number_of_stage):
	strat = ud.copy()
	return np.broadcast_to(strat, (number_of_stage, 2))

def mainly_nice(number_of_stage):
	strat = uc.copy()
	strat = np.broadcast_to(strat, (number_of_stage, 2)).copy()
	for i in range(number_of_stage):
		if random.random() >= probability_mainly_nice:
			strat[i, :] = ud
	return strat

def mainly_bad(number_of_stage):
	strat = ud.copy()
	strat = np.broadcast_to(strat, (number_of_stage, 2)).copy()
	for i in range(number_of_stage):
		if random.random() >= probability_mainly_bad:
			strat[i, :] = uc
	return strat

def tit_for_tat(opponent_strategy, number_of_stage):
	if opponent_strategy is tit_for_tat:
		return nice_guy(number_of_stage), nice_guy(number_of_stage)
	o = opponent_strategy(number_of_stage)
	return np.vstack((uc, o[:-1])), o

def nash_equilibrium(number_of_stage):
	strat = uc.copy()
	strat = np.broadcast_to(strat, (number_of_stage, 2)).copy()
	strat[number_of_stage-1, :] = ud
	return strat

def play (first_strat, second_strat, number_of_stage):
	if first_strat is tit_for_tat:
		f_strat, s_strat = first_strat(second_strat, number_of_stage)
	elif second_strat is tit_for_tat:
		s_strat, f_strat = second_strat(first_strat, number_of_stage)
	else:
		f_strat = first_strat(number_of_stage)
		s_strat = second_strat(number_of_stage)
	out_first = []
	out_second = []
	for f, s in zip(f_strat, s_strat):
		out_first += [f.dot(M).dot(s.T) , ]
		out_second += [s.dot(M).dot(f.T) , ]
	return np.array(out_first).sum(), np.array(out_second).sum()


In [114]:
strategies = [nice_guy, bad_guy, mainly_nice, mainly_bad, tit_for_tat, nash_equilibrium]
first_total_payoffs =np.zeros((len(strategies),len(strategies)))
second_total_payoffs =np.zeros((len(strategies),len(strategies)))

for i, f_strat in enumerate(strategies):
	for j, s_strat in enumerate(strategies):
		first_outcome, second_outcome = play(first_strat=f_strat, second_strat=s_strat, number_of_stage=n)
		first_total_payoffs[i, j] = first_outcome
		second_total_payoffs[i, j] = second_outcome
labels =['Nice Guy','Bad Guy','Mainly Nice', 'Mainly Bad', 'Tit for Tat', 'Nash equilibrium']

f = pd.DataFrame(first_total_payoffs, columns = labels , index= labels)
s = pd.DataFrame(second_total_payoffs, columns = labels , index= labels)
display(f)
display(s)

,Nice Guy,Bad Guy,Mainly Nice,Mainly Bad,Tit for Tat,Nash equilibrium
Nice Guy,2000.0,0.0,1532.0,470.0,2000.0,1998.0
Bad Guy,3000.0,1000.0,2494.0,1500.0,1002.0,2998.0
Mainly Nice,2249.0,254.0,1708.0,704.0,1768.0,2249.0
Mainly Bad,2747.0,739.0,2246.0,1281.0,1229.0,2763.0
Tit for Tat,2000.0,999.0,1729.0,1274.0,2000.0,1998.0
Nash equilibrium,2001.0,1.0,1545.0,517.0,2001.0,1999.0


,Nice Guy,Bad Guy,Mainly Nice,Mainly Bad,Tit for Tat,Nash equilibrium
Nice Guy,2000.0,3000.0,2234.0,2765.0,2000.0,2001.0
Bad Guy,0.0,1000.0,253.0,750.0,999.0,1.0
Mainly Nice,1502.0,2492.0,1762.0,2285.0,1768.0,1499.0
Mainly Bad,506.0,1522.0,755.0,1221.0,1226.0,471.0
Tit for Tat,2000.0,1002.0,1729.0,1277.0,2000.0,2001.0
Nash equilibrium,1998.0,2998.0,2226.0,2740.0,1998.0,1999.0
